In [8]:
from load_data import *
from binary_architecture import *
train_images, train_labels, test_images, test_labels = load_n_images(1200)
# make train_labels and test_labels binary: 0-4 -> 0, 5-9 -> 1
train_labels = np.array([0 if x < 5 else 1 for x in train_labels])
test_labels = np.array([0 if x < 5 else 1 for x in test_labels])

In [9]:
def one_cnn_run(args, seed, train_images, train_labels, test_images, test_labels):
    # Set seeds for reproducibility
    np.random.seed(seed)
    torch.manual_seed(seed)

    # Prepare data loaders
    train_dataset = TensorDataset(torch.Tensor(train_images).unsqueeze(1), torch.Tensor(train_labels).long())
    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)

    test_dataset = TensorDataset(torch.Tensor(test_images).unsqueeze(1), torch.Tensor(test_labels).long())
    test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

    # Instantiate models
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    encoder = Encoder(args).to(device)
    classifier = Class_out(args).to(device)
    conf_out = Conf_out(args).to(device)

    # Define loss functions and optimizer
    criterion_class = nn.CrossEntropyLoss()
    criterion_conf = nn.BCELoss()  # Binary Cross Entropy for confidence prediction
    optimizer = optim.Adam(list(encoder.parameters()) + list(classifier.parameters()) + list(conf_out.parameters()), lr=args.learning_rate)

    # Run the CNN denoise training loop
    best_model, test_z, test_conv_flat, stats = CNN_denoise(
        encoder=encoder,
        classifier=classifier,
        conf_out=conf_out,
        train_loader=train_loader,
        test_loader=test_loader,
        criterion_class=criterion_class,
        criterion_conf=criterion_conf,
        optimizer=optimizer,
        device=device
    )

    # Return or log the results
    results = {
        'best_model': best_model,
        'stats': stats
    }

    one_cnn_stats = stats
    print(one_cnn_stats)
    # Return the results
    return results, test_z, test_conv_flat

In [10]:
loop_num = 10

# Hyperparameters
class Args:
    latent_dim = 64 # 10, 32, 64, 128, 256... this would cause PCA testing accuracy to go up
    batch_size = 64
    epochs = 10
    learning_rate = 0.01

args = Args()

# Run the CNN denoise training loop
for seed in range(loop_num):
    results, _, _ = one_cnn_run(args, seed, train_images, train_labels, test_images, test_labels)

{'test_acc': 0.8332, 'test_conf': 0.7897, 'best_loss': 0.6048, 'best_acc': 0.8906, 'best_conf': 0.9184}
{'test_acc': 0.6767, 'test_conf': 0.9556, 'best_loss': 0.4113, 'best_acc': 0.9531, 'best_conf': 0.8685}
{'test_acc': 0.802, 'test_conf': 0.879, 'best_loss': 0.6792, 'best_acc': 0.875, 'best_conf': 0.8663}
{'test_acc': 0.8483, 'test_conf': 0.9328, 'best_loss': 0.2611, 'best_acc': 1.0, 'best_conf': 0.8178}
{'test_acc': 0.7128, 'test_conf': 0.9807, 'best_loss': 0.1701, 'best_acc': 1.0, 'best_conf': 0.9416}
{'test_acc': 0.7636, 'test_conf': 0.7797, 'best_loss': 0.2795, 'best_acc': 1.0, 'best_conf': 0.8794}
{'test_acc': 0.8588, 'test_conf': 0.9331, 'best_loss': 0.5104, 'best_acc': 0.9219, 'best_conf': 0.868}
{'test_acc': 0.8824, 'test_conf': 0.921, 'best_loss': 0.3363, 'best_acc': 1.0, 'best_conf': 0.7728}
{'test_acc': 0.8469, 'test_conf': 0.9053, 'best_loss': 0.5226, 'best_acc': 0.9062, 'best_conf': 0.9593}
{'test_acc': 0.8293, 'test_conf': 0.8989, 'best_loss': 0.287, 'best_acc': 1.0, 'b

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

train_pca, test_pca, train_reconstructed, test_reconstructed = PCA_reduction(train_images, test_images, args.latent_dim, random_seed=0)
# Logistic regression classifier for PCA-based latent space
clf_pca = LogisticRegression(max_iter=loop_num)
clf_pca.fit(train_pca, train_labels)  # Train on PCA latent space
test_preds_pca = clf_pca.predict(test_pca)  # Predict on test set
# Compute accuracy
acc_pca = accuracy_score(test_labels, test_preds_pca)
print(f"Accuracy on test set using PCA: {acc_pca * 100:.2f}%")

Accuracy on test set using PCA: 84.00%


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

def vote_majority(X):
    # along the second axis, find the most common element
    return np.array([Counter(column).most_common(1)[0][0] for column in X.T])

# Initialize a list to store the decision trees
decision_trees = []
prediction_trees = []
prediction_trees_probs = []

for i in range(loop_num): 
    # # Create a new decision tree
    tree = DecisionTreeClassifier(max_depth=3, random_state=i, max_features=8) # change max_features according to latent_dim
    # tree = RandomForestClassifier(n_estimators=loop_num, max_depth=5, random_state=i, max_features=64)
    # Fit the decision tree on the latent representations
    training_array = train_zs[i]
    tree.fit(training_array, train_labels)
    # Append the decision tree to the list
    decision_trees.append(tree)

    # Predict the test labels using the decision tree
    testing_array = test_zs[i]
    
    prediction_tree = tree.predict(testing_array)
    prediction_trees.append(prediction_tree)

    # Predict the test labels using the decision tree and calculate the probabilities
    prediction_tree_probs = tree.predict_proba(testing_array)
    prediction_trees_probs.append(prediction_tree_probs)

prediction_trees = np.array(prediction_trees)
prediction_trees_probs = np.array(prediction_trees_probs)

# vote for the majority
rf_predictions = vote_majority(prediction_trees)
# calculate the accuracy
print(f"Testing accuracy: {accuracy_score(test_labels, rf_predictions):.4f}")
# Calculate the average probability for each class across all trees
average_probs = np.mean(prediction_trees_probs, axis=0)
# Calculate the average confidence of the predictions
average_confidences = np.max(average_probs, axis=1)
average_confidence = np.mean(average_confidences)
print(f"Average confidence of predictions: {average_confidence:.4f}")

Testing accuracy: 0.1767
Average confidence of predictions: 0.1321


In [ ]:
# do the same analysis using XGBoost

from xgboost import XGBClassifier

# Initialize a list to store the XGBoost models
xgboost_models = []
prediction_xgboost = []
prediction_xgboost_probs = []

for i in range(loop_num):
    # Create a new XGBoost model
    xgboost = XGBClassifier(n_estimators=loop_num, random_state=i, learning_rate=0.1)
    # Fit the XGBoost model on the latent representations
    training_array = train_zs[i]
    xgboost.fit(training_array, train_labels)
    # Append the XGBoost model to the list
    xgboost_models.append(xgboost)

    # Predict the test labels using the XGBoost model
    testing_array = test_zs[i]
    prediction_xgb = xgboost.predict(testing_array)
    prediction_xgboost.append(prediction_xgb)

    # Predict the test labels using the XGBoost model and calculate the probabilities
    prediction_xgb_probs = xgboost.predict_proba(testing_array)
    prediction_xgboost_probs.append(prediction_xgb_probs)

prediction_xgboost = np.array(prediction_xgboost)
prediction_xgboost_probs = np.array(prediction_xgboost_probs)

# vote for the majority
xgboost_predictions = vote_majority(prediction_xgboost)
# calculate the accuracy
print(f"Testing accuracy: {accuracy_score(test_labels, xgboost_predictions):.4f}")
# Calculate the average probability for each class across all trees
average_probs = np.mean(prediction_xgboost_probs, axis=0)
# Calculate the average confidence of the predictions
average_confidences = np.max(average_probs, axis=1)
average_confidence = np.mean(average_confidences)
print(f"Average confidence of predictions: {average_confidence:.4f}")

Testing accuracy: 0.1333
Average confidence of predictions: 0.1234
